# Geospatial data analysis of time series
Quinn Brencher

In [ ]:
# Import required packages
import os
import numpy as np
import pandas as pd
import rasterio as rio
import matplotlib.pyplot as plt
import xarray as xr
import datetime as dt
import rioxarray
import seaborn as sns
import geopandas as gpd
from glob import glob
import gc
from matplotlib.dates import DateFormatter

## Load in and rebuild time series data

In [ ]:
# functions to load in mintpy data

# function to rewrite coordinates from metadata
def coord_range(ds):
    latrange = np.linspace(float(ds.attrs['Y_FIRST']),
                           ((float(ds.attrs['Y_STEP'])*float(ds.attrs['LENGTH']))+float(ds.attrs['Y_FIRST'])),
                           int(ds.attrs['LENGTH']))
    lonrange = np.linspace(float(ds.attrs['X_FIRST']),
                           ((float(ds.attrs['X_STEP'])*float(ds.attrs['WIDTH']))+float(ds.attrs['X_FIRST'])),
                           int(ds.attrs['WIDTH']))
    return latrange, lonrange

# function to read time series into xarray
def mintpyTS_to_xarray(fn, crs):
    ds = xr.open_dataset(fn, cache=False)
    ds = ds.rename_dims({'phony_dim_1':'time',
                         'phony_dim_2':'y',
                         'phony_dim_3':'x'})
    ds = ds.rename({'timeseries': 'displacement'})
    latrange, lonrange = coord_range(ds)
    ds = ds.assign_coords({'time': ('time', pd.to_datetime(ds.date)),
                           'y': ('y', latrange),
                           'x': ('x', lonrange)})
    ds = ds.drop(['bperp', 'date'])
    ds = ds.rio.write_crs(crs)
    
    return ds

def mintpy2d_to_xarray(fn, crs):
    ds = xr.open_dataset(fn)
    ds = ds.rename_dims({'phony_dim_0':'y',
                         'phony_dim_1':'x'
                        })
    latrange, lonrange = coord_range(ds)
    ds = ds.assign_coords({'y': ('y', latrange),
                           'x': ('x', lonrange)})
    ds = ds.rio.write_crs(crs)
    
    return ds

def mintpyInputs_to_xarray(fn, crs):
    ds = xr.open_dataset(fn)
    ds = ds.rename_dims({'phony_dim_0':'reference_time',
                         'phony_dim_1':'secondary_time',
                         'phony_dim_2':'y',
                         'phony_dim_3':'x'})
    latrange, lonrange = coord_range(ds)
    ds = ds.assign_coords({'reference_time': ('reference_time', pd.to_datetime([span[0] for span in ds.date.values])),
                           'secondary_time': ('secondary_time', pd.to_datetime([span[1] for span in ds.date.values])),
                           'y': ('y', latrange),
                           'x': ('x', lonrange)})
    ds = ds.drop('date')
    ds = ds.rio.write_crs(crs)
    
    return ds

In [ ]:
def open_summer_mintpy(orbit, year_list, suffix=''):
    ds_list = []
    home_path = '/mnt/d/indennt/mintpy_app'
    for year in year_list:
        mintpy_path = f'{home_path}/{orbit}/mintpy_{year}{suffix}'
        ts_ds = mintpyTS_to_xarray(f'{mintpy_path}/timeseries_ramp_demErr.h5', 32612)
        ts_ds = ts_ds.assign_coords({'year':('year', [int(year)])})
        ds_list.append(ts_ds)

    return ds_list

def load_aois(path, id):
    aoi_fn = f'{path}/rg{id}.shp'
    aoi_gdf = gpd.read_file(aoi_fn)

    stable_fn = f'{path}/stable{id}.shp'
    stable_gdf = gpd.read_file(stable_fn)

    ref_fn = f'{path}/ref{id}.shp'
    ref_gdf = gpd.read_file(ref_fn)

    return [aoi_gdf, stable_gdf, ref_gdf]

# change reference points
def change_ref(ds, rref):
    rref_ds = ds.copy(deep=True)
    rref_da = ds.displacement.sel(y=rref[1], x=rref[0], method='nearest')
    rref_ds['displacement'] = ds.displacement - rref_da
    return rref_ds

def change_ref_list(ds_list, rref):
    for i, ds in enumerate(ds_list):
        ds = change_ref(ds, rref)
        ds_list[i] = ds

    return ds_list

def crop_to_aois(ds_list, aoi_list):
    ds_list_aoi = []
    ds_list_stable = []
    ds_list_ref = []
    for ds in ds_list:
        print(f'working on {ds.year.item()}')
        ds_list_aoi.append(ds.rio.clip(aoi_list[0].geometry, crs=aoi_list[0].crs, drop=True))
        ds_list_stable.append(ds.rio.clip(aoi_list[1].geometry, crs=aoi_list[1].crs, drop=True))
        ds_list_ref.append(ds.rio.clip(aoi_list[2].geometry, crs=aoi_list[2].crs, drop=True))
        gc.collect()

    return [ds_list_aoi, ds_list_stable, ds_list_ref]

def mean_disp(ds_list_aoi):
    mean_list_aoi = []
    mean_list_stable = []
    mean_list_ref = []
    for ds in ds_list_aoi[0]:
        mean = ds.displacement.mean(dim=('y', 'x'))
        mean_list_aoi.append(mean)
    for ds in ds_list_aoi[1]:
        mean = ds.displacement.mean(dim=('y', 'x'))
        mean_list_stable.append(mean)
    for ds in ds_list_aoi[2]:
        mean = ds.displacement.mean(dim=('y', 'x'))
        mean_list_ref.append(mean)

    return [mean_list_aoi, mean_list_stable, mean_list_ref]

In [ ]:
orbit = 'AT137'
year_list = ['2017', '2018', '2019', '2020', '2021']

ds_list_CNN = open_summer_mintpy(orbit, year_list)
ds_list_uncorrected = open_summer_mintpy(orbit, year_list, suffix='_uncorrected')
# ds_list_murp = open_summer_mintpy(orbit, year_list, suffix='_MuRP')
# ds_list_era5 = open_summer_mintpy(orbit, year_list, suffix='_ERA5')

In [ ]:
aoi_list = load_aois('/mnt/d/indennt/polygons/aois', 5)

ds_list_aoi_CNN = crop_to_aois(ds_list_CNN, aoi_list)
ds_list_aoi_uncorrected = crop_to_aois(ds_list_uncorrected, aoi_list)

In [ ]:
mean_list_CNN = mean_disp(ds_list_aoi_CNN)
mean_list_uncorrected = mean_disp(ds_list_aoi_uncorrected)

## Displacement of aoi

In [ ]:
colors = ['red', 'orange', 'gold', 'green', 'blue', 'indigo', 'orchid']

sns.set_theme()
f, ax = plt.subplots(1, 2, figsize=(7.5,3), sharex=True, sharey=True)

ax[0].axhline(y=0, c='gray', alpha=0.5)
ax[1].axhline(y=0, c='gray', alpha=0.5)

# plot corrected aoi
for i, da in enumerate(mean_list_CNN[0]):
    da = da - mean_list_CNN[2][i]
    da['time'] = da.time.dt.strftime('2020-%m-%d').astype("datetime64[ns]") # remove year from date
    da = da - da.isel(time=0) # set starting displacement value to 0
    da = da*-100
    da.plot(ax=ax[0], label=year_list[i], c=colors[i],
            marker='o', markersize=4, linewidth=1, alpha=0.7)
    
# plot corrected stable area
for i, da in enumerate(mean_list_CNN[1]):
    da = da - mean_list_CNN[2][i]
    da['time'] = da.time.dt.strftime('2020-%m-%d').astype("datetime64[ns]") # remove year from date
    da = da - da.isel(time=0) # set starting displacement value to 0
    da = da*-100
    da.plot(ax=ax[1], label=year_list[i], c=colors[i],
            marker='o', markersize=4, linewidth=1, alpha=0.7)

date_form = DateFormatter("%m-%d")
ax[0].xaxis.set_major_formatter(date_form)

ax[0].set_title(None)
ax[1].set_title(None)
ax[0].set_ylabel('displacement (cm)')
ax[1].set_ylabel(None)
ax[0].set_xlabel(None)
ax[1].set_xlabel(None)
f.tight_layout()
#plt.savefig('../../figs/rg4-ref4_wref_CNN.png', dpi=300, bbox_inches='tight')
#ax.legend();

In [ ]:
sns.set_theme()
f, ax = plt.subplots(1, 2, figsize=(7.5,3), sharex=True, sharey=True)

ax[0].axhline(y=0, c='gray', alpha=0.5)
ax[1].axhline(y=0, c='gray', alpha=0.5)

# plot uncorrected aoi
for i, da in enumerate(mean_list_uncorrected[0]):
    da = da - mean_list_uncorrected[2][i]
    da['time'] = da.time.dt.strftime('2020-%m-%d').astype("datetime64[ns]") # remove year from date
    da = da - da.isel(time=0) # set starting displacement value to 0
    da = da*-100
    da.plot(ax=ax[0], label=year_list[i], c=colors[i],
            marker='o', markersize=4, linewidth=1, alpha=0.7)

# plot uncorrected stable area
for i, da in enumerate(mean_list_uncorrected[1]):
    da = da - mean_list_uncorrected[2][i]
    da['time'] = da.time.dt.strftime('2020-%m-%d').astype("datetime64[ns]") # remove year from date
    da = da - da.isel(time=0) # set starting displacement value to 0
    da = da*-100
    da.plot(ax=ax[1], label=year_list[i], c=colors[i],
            marker='o', markersize=4, linewidth=1, alpha=0.7)

date_form = DateFormatter("%m-%d")
ax[0].xaxis.set_major_formatter(date_form)

ax[0].set_title(None)
ax[1].set_title(None)
ax[0].set_ylabel('displacement (cm)')
ax[1].set_ylabel(None)
ax[0].set_xlabel(None)
ax[1].set_xlabel(None)
f.tight_layout()

#plt.savefig('../../figs/rg4-ref4_wref_uncorrected.png', dpi=300, bbox_inches='tight')
#ax.legend();